# 2022 Jira Jupyter Gephi Disco

Based on the examples in https://towardsdatascience.com/communication-story-from-an-issue-tracking-software-efbbf29736ff
    

In [ ]:
# Import dependencies
from jira import JIRA, JIRAError
from collections import Counter, defaultdict
from datetime import datetime
from time import sleep

import numpy as np
import pandas as pd
import networkx as nx


# Create instance for interacting with Jira
jira = JIRA(options={'server': url}, basic_auth=(username, password))


# Read data from Jira with changelog
jira_search = jira.search_issues(jql, startAt=block_num*block_size, maxResults=block_size, 
		fields="issuetype, created, resolutiondate, reporter, assignee, status", 
		expand='changelog')


# Define parameters for writing data
index_beg = 0
header = True
mode = 'w'

# Iteratively read data
while bool(jira_search):
# Container for Jira's data
	data_jira = []

	for issue in jira_search:
# Get issue key
	issue_key = issue.key

# Get request type
request_type = str(issue.fields.issuetype)

# Get datetime creation
	datetime_creation = issue.fields.created
	if datetime_creation is not None:
# Interested in only seconds precision, so slice unnecessary part
	datetime_creation = datetime.strptime(datetime_creation[:19], "%Y-%m-%dT%H:%M:%S")

# Get datetime resolution
	datetime_resolution = issue.fields.resolutiondate
	if datetime_resolution is not None:
# Interested in only seconds precision, so slice unnecessary part
	datetime_resolution = datetime.strptime(datetime_resolution[:19], "%Y-%m-%dT%H:%M:%S")

# Get reporter’s login and name
	reporter_login = None
	reporter_name = None
	reporter = issue.raw['fields'].get('reporter', None)
	if reporter is not None:
	reporter_login = reporter.get('key', None)
	reporter_name = reporter.get('displayName', None)

# Get assignee’s login and name
	assignee_login = None
	assignee_name = None
	assignee = issue.raw['fields'].get('assignee', None)
	if assignee is not None:
	assignee_login = assignee.get('key', None)
	assignee_name = assignee.get('displayName', None)

# Get status
	status = None
	st = issue.fields.status
	if st is not None:
	status = st.name

# Read data from Jira with changelog
	jira_search = jira.search_issues(jql, startAt=block_num*block_size, maxResults=block_size, 
			fields="issuetype, created, resolutiondate, reporter, assignee, status", 
			expand='changelog')

# Get information from changelog
	history_assignee = []
	histories = issue.raw['changelog'].get('histories', None)
	if histories is not None:
	for history in histories:
	for item in history['items']:
	if item['field'] == 'assignee':
# Get history author, previous assignee, new assignee
	history_author = history.get('author', None)
	if history_author is not None:
	history_author = history_author['key']
	history_assignee.append([history_author, item['from'], item['to'], datetime.strptime(history['created'][:19], "%Y-%m-%dT%H:%M:%S")])

# Add data to data_jira
data_jira.append((issue_key, request_type, datetime_creation, datetime_resolution, reporter_login, reporter_name, assignee_login, assignee_name, status))

# Write data read from Jira
index_end = index_beg + len(data_jira)
	data_jira = pd.DataFrame(data_jira, index=range(index_beg, index_end), 
			columns=['Issue key', 'Request type', 'Datetime creation', 'Datetime resolution', 'Reporter login', 'Reporter name', 'Assignee login', 'Assignee name', 'Status'])
	data_jira.to_csv(path_or_buf='data_jira.csv', sep=';', header=header, index=True, index_label='N', mode=mode)

# Update for the next iteration
	block_num = block_num + 1
	index_beg = index_end
	header = False
	mode = 'a'

# Print how many issues were read
	if block_num % 50 == 0:
print(block_num * block_size)

# Pause before next reading – it’s optional, just to be sure we will not overload Jira’s server
sleep(1)

# New issues search
	jira_search = jira.search_issues(jql, startAt=block_num*block_size, maxResults=block_size, 
			fields="issuetype, created, resolutiondate, reporter, assignee, status")

jira.close()
	except (JIRAError, AttributeError):
		jira.close()
		print('Error')



# Read data from created before csv file
reporter_assignee = defaultdict(int)
creation = {}
chunksize = 1000
for chunk in pd.read_csv('data_jira.csv', sep=';', header=0, usecols=['Reporter login', 'Assignee login', 'Datetime creation'], engine='python', chunksize=chunksize, encoding='utf_8_sig'):
    for i in chunk.index:
        reporter = chunk.loc[i, 'Reporter login']
        assignee = chunk.loc[i, 'Assignee login']
        data_beg = datetime.strptime(chunk.loc[i, 'Datetime creation'], "%Y-%m-%d %H:%M:%S").date()
        if reporter is not np.nan and assignee is not np.nan: 
            reporter_assignee[(reporter, assignee)] += 1
            if creation.get(reporter, None) is None:
                creation[reporter] = {'creation': data_beg}
            else:
                if data_beg < creation[reporter]['creation']:
                    creation[reporter] = {'creation': data_beg}
            if creation.get(assignee, None) is None:
                creation[assignee] = {'creation': data_beg}
            else:
                if data_beg < creation[assignee]['creation']:
                    creation[assignee] = {'creation': data_beg}

# Create graph
graph_units = nx.DiGraph()
graph_units.add_weighted_edges_from([(x[0], x[1], y) for x, y in reporter_assignee.items()])

# Set for every node date creation
for item in creation:
    creation[item]['creation'] = creation[item]['creation'].strftime("%Y-%m-%d")
nx.set_node_attributes(graph_units, creation)

# Set a color for nodes according business units (business units read from special file, so this code may be dropped)
unit_colors = {}
for item in Counter(active_dir['NAME_UNIT']).keys():
    unit_colors[item] = '#{:02x}{:02x}{:02x}'.format(random.randint(0,240), random.randint(0,255), random.randint(0,255))
colors = {}
for item in graph_units.nodes:
    employee = active_dir[active_dir['LOGIN'] == item]['IBLOCK_SECTION_NAME_2']
    if not employee.empty:
        if employee.values[0] is not np.nan:
            colors[item] = {'color': unit_colors[employee.values[0]]}
        else:
            colors[item] = {'color': '#ff0000'} # #ff0000 - red color
    else:
        colors[item] = {'color': '#ff0000'}
nx.set_node_attributes(graph_units, colors)

# Save graph
nx.write_graphml(graph_units, 'graph_employee.graphml')



Now go get Gephi and/or Disco to load the two data files:

- data_jira.csv into Disco
- graph_employee.graphml into Gephi